In [ ]:
import requests
import pandas as pd
import numpy as np


# https://martin-thoma.com/configuration-files-in-python/

import json

with open("../.private/keys.json") as keys_file:
    KEYS = json.load(keys_file)

APIKEY = KEYS['THEGRAPH']
# Note: don't print the key, or if you do, delete the cell's output
# (cell outputs are saved and can be sent to Github).

Badger DAO graph endpoint.

In [ ]:
# Badger DAO
ENDPOINT = "https://gateway.thegraph.com/api/" + APIKEY + \
    "/subgraphs/id/GGv13zuXnvG4V97Yn8FvfY5c73j1Df78D9rzvSye3yXr"
# https://thegraph.com/explorer/subgraph?id=GGv13zuXnvG4V97Yn8FvfY5c73j1Df78D9rzvSye3yXr&view=Playground
    

Load the gql client.

In [ ]:
# !pip install gql
# !pip install aiohttp 
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport


In [ ]:
transport = AIOHTTPTransport(url=ENDPOINT)
client = Client(
    transport=transport
    #fetch_schema_from_transport=True
)


Let's try a first query.

In [ ]:
query = gql(
"""
{
  users(first: 5) {
    id
    settBalances {
      id
    }
  }
  setts(first: 5) {
    id
    name
    symbol
    token {
      id
    }
  }
}
""")

In [ ]:
result = await client.execute_async(query)
print(result)



In [ ]:
result['users']

In [ ]:
result['setts']

In [ ]:
query = gql(
"""
  query ($first: Int!, $skip: Int!) {
    users(
        first: $first,
        skip: $skip
    ) {
        id
        settBalances {
        id
        netDeposit
        }
    }
}
""")

In [ ]:
users_all = []

In [ ]:
users = users_all
first = 1000
skip = len(users_all)
fetch = True
while fetch:
    vars = {"first": first, "skip": skip}
    res = await client.execute_async(query, variable_values=vars)
    # print(type(res))
    # print(res)
    
    if not res['users']:
        print('I am done!')
        fetch = False
    else:
        users.extend(res['users'])
        print(len(users))
        skip += first# fetch = False

In [ ]:
len(users)

In [ ]:
df = pd.DataFrame(users)

In [ ]:
df.info()

In [ ]:
aa = df['id'].duplicated()
aa.sum()

In [ ]:
df.loc[1233]['settBalances']